# Choosing your Houseplant

Makes sense! One thing- for toxicity, I think it should be 0 toxicity vs any toxicity. I think probably you wouldn't want a cat or child eating a plant that even had a 1 toxicity level

Feel free to look at Project_sam.ipynb on our github page- under spring2019-group-project-team_plant/src/myproject/Plant Project_sam.ipynb
-- Samantha 





NOTE: I decided to exclude pests? and the soil type? questions in the user interaction because pests would eliminate too many possible choices and the soil type is too varied-- we can just make these as notes that plants tend to get pests and here's how to avoid. and we can make a note of "how to take care of the plant-- pruning, propagation, and soil type, with everything else" -Jessica




In [0]:
# Import (Not sure what we will need from these yet)

import os, sys, csv, random
#import pydicom
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import seaborn as sns
import pandas as pd
from PIL import Image
from ipywidgets import widgets
from IPython.display import display

In [2]:
# Load in the dataset- this is the sample dataset that was downloaded as CSV from google sheets and uploaded to our group's github page
# To get the path -> click "raw" and then copy the url
path_to_csv = 'https://raw.githubusercontent.com/biof309/spring2019-group-project-team_plant/master/src/myproject/Plant%20Dataset%20-%20Data%20(2).csv'
plant_ds = pd.read_csv(path_to_csv)
plant_ds.head()

,ID,Plant,Plant taxonomy,PlantSize,Light,Water,Temp_low (degF),Temp_high (degF),Temp_Range,TempDurability,...,Flowering,Pests,Soil,PotSize,Pruning,Propogation (multiple options),SpecialOccasion,CleanAirPlant,Toxicitylevel,"Verified (1 = good to go, 0 = possible error)"
0,NaN,NaN,NaN,"(small < 1 ft, medium, large > 3 ft)","(low, medium, high)","(low, medium, high)",NaN,NaN,NaN,NaN,...,"(yes, no, small, rare)","(yes, no)",(multiple options),"(small, medium, large)","(regular, as-needed, none)",NaN,"(yes, no)",(yes/no),NaN,NaN
1,1.0,African Violet,Saintpaulia ionantha,NaN,medium,medium,75.0,80.0,5.0,low,...,yes,yes,"rich, airy",small,regular,leaf cutting,yes,no,0.0,NaN
2,2.0,Agave Plant,Agave attenuata,NaN,high,low,50.0,80.0,30.0,high,...,rare,yes,"quick draining, cactus",small,none,offset,no,no,2.0,NaN
3,3.0,Alocasia,Alocasia Amazonica,NaN,medium,medium,60.0,80.0,20.0,medium,...,small,yes,"organic, well-aerated, loose",small,as-needed,plant division,no,no,4.0,NaN
4,4.0,Alocasia-Nebula,Alocasia nebula imperialis,NaN,medium,medium,60.0,80.0,20.0,medium,...,small,yes,"organic, well-aerated, loose",small,as-needed,plant division,no,no,4.0,NaN


In [0]:
# List of Questions and Options- will have to finish filling in from the google doc
# Important! As of right now, I can't figure out how to allow a different number of options for each quetion,
# so we will have to limit the options to 3

questionlist = [
    "What size plant are you looking for?",
    "What size pot do you prefer?",
    "How much light will your indoor plant have?",
    "Do you want a plant that cleans the air?", #This is for clean air plant
    "Do you have a child or pet?", #This is for toxicity
    "Did you want this plant to flower?", 
    "How much temperature fluctuation will this plant encounter?", #This is for plant durability
    "Did you want to give this plant for a special occasion?",
    "How much work do you want to do on this plant?", #This is for pruning
    "Do you tend to underwater or overwater?",
    "How humid is your home?"
    
]

optionlist = [
    ["Small", "Medium", "Large"],
    ["Small", "Medium", "Large"],
    ["Low","Medium","High"], 
    ["Yes","No", "Does not matter"],#doesn't matter=yes and no
    ["Yes", "No", "Does not matter"], #Yes= toxicity is 0, No= all other ones
    ["Yes", "No","Does not matter"], #doesn't matter=everything 
    ["Low", "Medium", "High"], 
    ["Yes", "No", "Does not matter"], #doesn't matter=everything
    ["Minimal","Lots","Does not matter"], #minimal=none, lots=regular, doesn't matter=anything
    ["Underwater","Overwater","No"], #underwater=low, overwater=high, no=anything
    ["Low","Medium","High"]
]

In [0]:
# GUI code for quiz questions:

# Style from: https://github.com/abhijitnathwani/PyQuiz/blob/master/build/lib.linux-x86_64-2.7/py_quiz/__main__.py

import tkinter as tk
import tkinter.messagebox
from ipywidgets import widgets
from IPython.display import display

def main(args=None):
    app = PlantQuiz() # creating the object for Application class()
    app.master.title('Plant Quiz')
    app.mainloop()
    return app

class PlantQuiz(tk.Frame):
    #(tk.Frame)
    def __init__(self, master=None):
        tkinter.messagebox.showinfo('Welcome!','Are you ready to find out which house plant best suits you??')
        tk.Frame.__init__(self, master)
        self.flag=0
        self.qn = 0
        self.anslist = [None]*len(questionlist)
        self.grid()
        # declaring variables to store question and answer
        self.optionA = tk.StringVar() # control variable for option A
        self.optionB = tk.StringVar() # control variable for option B
        self.optionC = tk.StringVar() # control variable for option C
        self.selected_answer = tk.StringVar() # variable to get the selected answer
        self.question = tk.StringVar() # control variable for the question to be loaded
        self.questions = questionlist
        top = self.winfo_toplevel()
        self.createWidgets(top) # call to create the necessary widgets
        self.load_question(top) # load the first question

    def new_game(self,top):
        self.load_question(top)
        
#     def about(self):
#         tkinter.messagebox.showinfo("About Plant Quiz","Welcome to Plant Quiz")

    def confirm_quit(self):
        choice = tkinter.messagebox.askyesno('Quit the Quiz','Do you really want to quit?')
        if choice == True:
            self.destroy()
        elif choice == False:
            pass

    def set_ans(self,answer):
        if answer==1:
            self.selected_answer = self.optionA.get()
        elif answer==2:
            self.selected_answer = self.optionB.get()
        elif answer == 3:
            self.selected_answer = self.optionC.get()
        self.flag=1

    def store_ans(self):
    
        self.anslist[self.qn] = self.selected_answer
        print(str(self.selected_answer))

    def load_question(self,top):
        
        self.radioButtonA.select() # sets the first radio button as the default
        self.radioButtonA.deselect()
        
        self.answers = optionlist[self.qn]
        self.question.set(questionlist[self.qn])
        #length=len(self.question.get())  # get the length of the question
        #width=str(100+10*length)
        width=str(500) #500
        height=str(300) #180
        top.geometry(width+"x"+height)
        
        self.optionA.set(optionlist[self.qn][0])
        self.optionB.set(optionlist[self.qn][1])
        self.optionC.set(optionlist[self.qn][2])

    def next_btn(self, top):
        #print("self.qn: ", self.qn)
        self.store_ans()
        
        if self.qn >= (len(questionlist)-1):
            self.store_ans()
            tkinter.messagebox.showinfo('Bye!','You are finished! Click ok to calculate results.')
            self.destroy()
        else: 
            self.qn = self.qn + 1
            self.load_question(top)
 
    def createWidgets(self,top):

        #top.geometry("500x300")
        #top.geometry(width+"x"+height)
        top.resizable(True,True)
        top.grid_columnconfigure(0,weight=1)
        top.grid_columnconfigure(9,weight=1)
        top.grid_rowconfigure(0,weight=1)
        top.grid_rowconfigure(9,weight=1)

        #Creating the menu buttons
#         self.menu = tk.Menu(self)
#         self.menubar = tk.Menu(self.menu, tearoff=0)
#         self.menubar.add_command(label="New Game", command=lambda: self.new_game(top))
#         self.menubar.add_command(label="About", command=self.about)
#         self.menubar.add_command(label="Quit", command=self.confirm_quit)
#         top.config(menu=self.menubar)

        #Creating the buttons
        self.quitButton = tk.Button(self, text='Quit', command=self.confirm_quit)
        self.nextButton = tk.Button(self, text='Next', command=lambda: self.next_btn(top))

        #Creating Radio buttons for options
        self.radioButtonA = tk.Radiobutton(self,anchor='w', 
                            textvariable=self.optionA, 
                            variable = self.selected_answer, 
                            value = 'A',
                            command = lambda: self.set_ans(1)) # the radio button call 'set_ans()' with the number to set the 'selected_answer' variable
        self.radioButtonB = tk.Radiobutton(self,anchor='w',
                            textvariable=self.optionB, 
                            variable = self.selected_answer,
                            value = 'B', 
                            command = lambda: self.set_ans(2))
        self.radioButtonC = tk.Radiobutton(self,anchor='w',
                            textvariable=self.optionC, 
                            variable = self.selected_answer, 
                            value = 'C', 
                            command = lambda: self.set_ans(3))

        #Creating the labels for options and questions
            
        self.label_question = tk.Label(self,textvariable=self.question)
        
        #Packing the widgets in the grid

        self.label_question.grid(column=3,row=1,columnspan=4)
        self.radioButtonA.grid(column=4,row=2, columnspan=3,sticky=tk.N+tk.S+tk.W+tk.E)
        self.radioButtonB.grid(column=4,row=3, columnspan=3,sticky=tk.N+tk.S+tk.W+tk.E)
        self.radioButtonC.grid(column=4,row=4, columnspan=3,sticky=tk.N+tk.S+tk.W+tk.E)
            
        self.quitButton.grid(column=6,row=5) #,sticky=tk.N+tk.S+tk.W+tk.E)
        self.nextButton.grid(column=3,row=5) #,sticky=tk.N+tk.S+tk.W+tk.E)
        
if __name__ == "__main__":
    myquiz = main()
    
# The variable that saves the quiz answers is called myquiz.anslist:
print(list(myquiz.anslist))


TclError: ignored

In [0]:
#test_answers = myquiz.anslist # This will be the actual code
test_answers = ['Medium', "Large","High","No","Yes","Does not matter","Low","Yes","Minimal","Overwater","Low"] # This is just to use for reference so you don't have to take the quiz every time


In [0]:
#Me trying to solve the issue below
plant_sizeds=plant_ds.PlantSize
test_answers[0]=='Small'

a=plant_sizeds.loc(0,'small')

TypeError: ignored

In [0]:
# Sample test list of answers to play around with, which will be generated by the GUI
# 0 indicates the first option

#sorry! having trouble selecting the specific element and saving it into a list with the plant name
plant_sizeds=plant_ds.PlantSize #Plant size dataset from the excel sheet 
if test_answers[0]=='Small'
  plant_size=(plant_sizeds('small')) #these have issues
elif test_answers[0]=='Medium'
  plant_size=(plant_sizeds('medium')) #these have issues
else test_answers[0]='Large'
  plant_size=(plant_sizeds('large')) #these have issues continued down 
  
potsize_ds=plant_ds.PotSize
if test_answers[1]=='Small'
  pot_size=(potsize_ds('small'))
elif test_answers[1]=='Medium'
  pot_size=(potsize_ds('medium'),potsize_ds('regular'))
else test_answers[1]=='Large'
  pot_size=(potsize_ds('large'))
  
light_ds=plant_ds.Light
if test_answers[2]=='Low'
   plant_light=(light_ds('low'))
elif test_answers[2]=='Medium'
  plant_light=(light_ds('medium'))
else test_answers[2]=='High'
  plant_light=(light_ds('high'))
  
cleanair_ds=plant_ds.CleanAirPlant
if test_answers[3]=='Yes'
  cleanair=(cleanair_ds('yes'))
elif test_answers[3]=='No'
  cleanair=(cleanair_ds('no'))
else test_answers[3]=='Does not matter'
  cleanair=(cleanair_ds('yes'), cleanair_ds('no'))

toxicity_ds=plant_ds.Toxicitylevel
if test_answers[4]=='Yes'
  toxicity=(toxicity_ds==0)
else test_answers[4]=='No'
  toxicity=(toxicity>0)

flower_ds=plant_ds.Flowering
if test_answers[5]=='Yes'
  flower=(flower_ds('yes'),flower_ds('small'))
elif test_answers[5]=='No'
  flower=(flower_ds('no'), flower_ds('rare'))
else test_answers[5]=='Does not matter'
  flower=flower_ds
  
tempdurability_ds=plant_ds.TempDurability
if test_answers[6]=='Low'
  durability=(tempdurability_ds('low'))
elif test_answers[6]=='Medium'
  durability=(tempdurability_ds('medium'))
else test_answers[6]=='High'
  durability=(tempdurability_ds('high'))

ocassion_ds=plant_ds.SpecialOccasion
if test_answers[7]=='Yes'
  occasion=(occasion_ds('yes'))
elif test_answers[7]=='No'
  occasion=(occasion_ds('no'))
else test_answers[7]=='Does not matter'
  occasion=ocassion_ds


#numberofplants= #number of plants selected based on user input to convert to a score
#import random
#listofplants=random.sample(numberofplants,5)
#print("Here are the plants that may fit you:" listofplants)

#need to convert this GUI output to select items from the plant dataset
#after converting the GUI output answers to the plant dataset answers, need to output a number with the score
#randomly select 5 plants from the ones that GUI has narrowed down
#output those options to the user with notes about soil type (tell python to output that cell with our soil info), pruning (from cell), propagation (From cell), and a general note about pests




SyntaxError: ignored

IDEA:

For each of the answers, we *score* the plants in the entire dataset against the answers, so we will essentially generate a matrix of 1's (for yes the answer matches) and 0's (for no, the answer does not match).
Then whichever plant has the most 1's, that is the plant that is the best match.

For example, if a person's quiz answers are:
test_answers = ["Medium", "No","Low","Yes","Yes","Yes","Yes","Yes","Minimal","No","Normal"] 

Then for the first 3 plants, the match might look like (and I'm just making this up):

[0, 1, 1, 0, 1, 1, 0   (For the African Violet)

0, 0, 0, 1, 0, 1, 0     (for the Agave Plant)

0, 1, 0 , 0, 0, 1, 1]  (for the Alocasia)

So when we sum up the rows, we'd get:
african violet = 4
agave plant = 2
alocasia = 3
The african violet would be the best match, with a 4/7, or 57% match

# Plant Selection Mechanism

This is what I have created so far following Samantha's matrix concept...

~Stuart

In [0]:
# Converting user response list to dataframe

user_resp = ["large", "large", "high", "does not matter", "yes", "yes", "low", "yes", "minimal", "underwater", "low"]

user_resp_df = pd.DataFrame(np.array(user_resp).reshape(1,11))

In [7]:
# Creating a 1's and 0's dataframe

    # Question 1

plant_ds['PlantSize'] = (plant_ds['PlantSize'] == str(user_resp_df.loc[0,0])).astype(int)

    # Question 2

plant_ds['PotSize '] = (plant_ds['PotSize '] == str(user_resp_df.loc[0,1])).astype(int)

    # Question 3
  
plant_ds['Light '] = (plant_ds['Light '] == str(user_resp_df.loc[0,2])).astype(int)

    # Question 4 - gonna have to change plant_ds to match user responses - right now "does not matter" does not line up with yes or no
  
plant_ds['CleanAirPlant'] = (plant_ds['CleanAirPlant'] == str(user_resp_df.loc[0,3])).astype(int)

    # Question 5 - same problem as Q4
  
plant_ds['Toxicitylevel'] = (plant_ds['Toxicitylevel'] == str(user_resp_df.loc[0,4])).astype(int)

    # Question 6
  
plant_ds['Flowering '] = (plant_ds['Flowering '] == str(user_resp_df.loc[0,5])).astype(int)

    # Question 7

plant_ds['TempDurability'] = (plant_ds['TempDurability'] == str(user_resp_df.loc[0,6])).astype(int)

    # Question 8
  
plant_ds['SpecialOccasion '] = (plant_ds['SpecialOccasion '] == str(user_resp_df.loc[0,7])).astype(int)

    # Question 9

plant_ds['Pruning '] = (plant_ds['Pruning '] == str(user_resp_df.loc[0,8])).astype(int)

    # Question 10
  
plant_ds['Water '] = (plant_ds['Water '] == str(user_resp_df.loc[0,9])).astype(int)

    # Question 11
  
plant_ds['Humidity '] = (plant_ds['Humidity '] == str(user_resp_df.loc[0,9])).astype(int)

plant_ds.head(30)
  

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,ID,Plant,Plant taxonomy,PlantSize,Light,Water,Temp_low (degF),Temp_high (degF),Temp_Range,TempDurability,...,Flowering,Pests,Soil,PotSize,Pruning,Propogation (multiple options),SpecialOccasion,CleanAirPlant,Toxicitylevel,"Verified (1 = good to go, 0 = possible error)"
0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,...,0,"(yes, no)",(multiple options),0,0,NaN,0,0,0,NaN
1,1.0,African Violet,Saintpaulia ionantha,0,0,0,75.0,80.0,5.0,1,...,1,yes,"rich, airy",0,0,leaf cutting,1,0,0,NaN
2,2.0,Agave Plant,Agave attenuata,0,1,0,50.0,80.0,30.0,0,...,0,yes,"quick draining, cactus",0,0,offset,0,0,0,NaN
3,3.0,Alocasia,Alocasia Amazonica,0,0,0,60.0,80.0,20.0,0,...,0,yes,"organic, well-aerated, loose",0,0,plant division,0,0,0,NaN
4,4.0,Alocasia-Nebula,Alocasia nebula imperialis,0,0,0,60.0,80.0,20.0,0,...,0,yes,"organic, well-aerated, loose",0,0,plant division,0,0,0,NaN
5,5.0,Aloe Vera Plant,Aloe Vera,0,1,0,65.0,85.0,20.0,0,...,1,no,any,0,0,"offset, sucker",0,0,0,NaN
6,6.0,Aluminum Plant,Pilea,0,0,0,60.0,75.0,15.0,0,...,0,yes,airy,0,0,stem cutting,0,0,0,NaN
7,7.0,Amaryllis,Hippeastrum,0,0,0,65.0,70.0,5.0,1,...,1,yes,"bulb, quick draining",0,0,offset,1,0,0,NaN
8,8.0,Angel Wing Begonia,Begonia Coccinea,0,0,0,65.0,75.0,10.0,1,...,1,yes,rich,0,0,stem cutting,0,0,0,NaN
9,9.0,Anthurium,Anthurium,0,0,0,75.0,85.0,10.0,1,...,1,yes,"rich, organic, loose",0,0,"stem cutting, offset",0,0,0,NaN


# Current Problem

Alright the 1's and 0's concept works

We need to make sure that the user responses correspond to values in the plant_ds... this works for some columns where the responses are "low" or "medium"... but for when user responses are "does not matter", there needs to be code that takes this into account

In [8]:
list(plant_ds.columns.values)

['ID',
 'Plant',
 'Plant taxonomy',
 'PlantSize',
 'Light ',
 'Water ',
 'Temp_low (degF)',
 'Temp_high (degF)',
 'Temp_Range',
 'TempDurability',
 'Humidity ',
 'Flowering ',
 'Pests ',
 'Soil ',
 'PotSize ',
 'Pruning ',
 'Propogation (multiple options)',
 'SpecialOccasion ',
 'CleanAirPlant',
 'Toxicitylevel',
 'Verified (1 = good to go, 0 = possible error)']

In [0]:
# Delete all the columns that we aren't using in the dataframe just to clean it up

slim_plant_df = plant_ds[["Plant", "Plant taxonomy", "PlantSize", "Light ", "Water ", "TempDurability", "Humidity ", "Flowering ", "PotSize ", "Pruning ", "SpecialOccasion ", "CleanAirPlant", "Toxicitylevel"]]

In [10]:
# Create a new column in this new dataset that sums all the ones and zeros
slim_plant_df.loc[:, 'total'] = slim_plant_df.iloc[:, 1:].sum(axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
# Sort dataset by descending numbers of the "sum column"

final_df = slim_plant_df.sort_values(by=['total'], ascending = False)

In [12]:
# View final_df

final_df.head()

,Plant,Plant taxonomy,PlantSize,Light,Water,TempDurability,Humidity,Flowering,PotSize,Pruning,SpecialOccasion,CleanAirPlant,Toxicitylevel,total
109,Poinsettia Plant,Euphorbiaceae pulcherrima,1,1,0,1,0,1,1,0,1,0,0,6
63,Fatsia Plant,Fatsia japonica,1,1,0,0,0,1,1,0,0,0,0,4
128,Stromanthe Tricolor Plant,Stromanthe sanguinea,1,1,0,1,0,1,0,0,0,0,0,4
135,Zebra Plant,Aphelandra squarrosa,1,1,0,1,0,1,0,0,0,0,0,4
1,African Violet,Saintpaulia ionantha,0,0,0,1,0,1,0,0,1,0,0,3


In [37]:
# Display the top 5 plants or whatever we want

print("Your Top 5 plants are: ")

for re in range(0,5):

  print(str(re+1) + ". "+str(final_df.iloc[re,0])+" ("+str(round(100*(final_df.iloc[re,13]/11),1))+"%)")
  #print("2. " + str(final_df.iloc[1,0]) + )
  #print("3. " + str(final_df.iloc[2,0]) + )
  #print("4. " + str(final_df.iloc[3,0]) + )
  #print("5. " + str(final_df.iloc[4,0]) + )

Your Top 5 plants are: 
1. Poinsettia Plant (54.5%)
2. Fatsia Plant (36.4%)
3. Stromanthe Tricolor Plant (36.4%)
4. Zebra Plant (36.4%)
5. African Violet (27.3%)
